In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torch.utils.data import random_split, DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import SGD


In [ ]:
CIFAR_10 = '/content/drive/MyDrive/Deep Learning/CIFAR-10'
training_dataset = CIFAR10(root=CIFAR_10, transform=ToTensor(), download=False, train=True)
testing_dataset = CIFAR10(root=CIFAR_10, transform=ToTensor(), download=False, train=False)

In [ ]:
print(len(training_dataset))
print(len(testing_dataset))

50000
10000


In [ ]:
train_ds, val_ds = random_split(training_dataset, [45000, 5000])
train_dl = DataLoader(train_ds, batch_size=128, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=128, shuffle=False)
test_dl = DataLoader(testing_dataset, batch_size=128, shuffle=False)

In [ ]:
class CIFARModel(nn.Module):
    def __init__(self):
        super(CIFARModel, self).__init__()
        self.flatten = nn.Flatten()
        self.layer1 = nn.Linear(3 * 32 * 32, 512)
        self.layer2 = nn.Linear(512, 256)
        self.layer3 = nn.Linear(256, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 3 * 32 * 32)
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.relu(x)
        x = self.layer3(x)
        return x
model = CIFARModel()

In [ ]:
model.state_dict()

In [ ]:
def train(epochs, model, loss_fn, opt, train_dl, val_dl, device):
  epochs = 100
  for epoch in range(epochs) :

        print(f'Epoch {epoch+1}/{epochs}')
        model.train()
        for input, target in train_dl:
            input = input.to(device)
            target = target.to(device)
            pred = model(input)
            loss = loss_fn(pred, target)
            loss.backward()
            opt.step()
            opt.zero_grad()


        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for input, target in val_dl:
                input = input.to(device)
                target = target.to(device)
                pred = model(input)
                _, predicted = torch.max(pred.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        print(f'Train Loss: {loss.item():.4f}, Validation Accuracy: {(100 * correct / total):.2f}%')

        return model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CIFARModel().to(device)
loss_fn = CrossEntropyLoss()
opt = SGD(model.parameters(), lr=0.45)


model.eval()
correct = 0
total = 0
with torch.no_grad():
    for input, target in test_dl:
        input = input.to(device)
        target = target.to(device)
        pred = model(input)
        _, predicted = torch.max(pred.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f'Test Accuracy: {(100 * correct / total):.2f}%')


def train(num_epochs, model, loss_fn, optimizer, train_loader, val_loader, device):
    train_losses = []
    val_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        train_loss = total_loss / len(train_loader)
        train_losses.append(train_loss)

        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        val_accuracies.append(val_accuracy)

        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    return model, train_losses, val_accuracies

Test Accuracy: 9.58%


In [ ]:
trained_model = train(10, model, loss_fn, opt, train_dl, val_dl, device)

Epoch [1/50], Train Loss: 7.0934, Validation Accuracy: 9.56%
Epoch [2/50], Train Loss: 4.4527, Validation Accuracy: 10.06%
Epoch [3/50], Train Loss: 3.2268, Validation Accuracy: 10.02%
Epoch [4/50], Train Loss: 2.5368, Validation Accuracy: 9.90%
Epoch [5/50], Train Loss: 2.3059, Validation Accuracy: 9.96%
Epoch [6/50], Train Loss: 2.3029, Validation Accuracy: 10.48%
Epoch [7/50], Train Loss: 2.3029, Validation Accuracy: 9.76%
Epoch [8/50], Train Loss: 2.3029, Validation Accuracy: 10.02%
Epoch [9/50], Train Loss: 2.3029, Validation Accuracy: 9.56%
Epoch [10/50], Train Loss: 2.3029, Validation Accuracy: 9.76%
Epoch [11/50], Train Loss: 2.3029, Validation Accuracy: 10.02%
Epoch [12/50], Train Loss: 2.3029, Validation Accuracy: 9.76%
Epoch [13/50], Train Loss: 2.3029, Validation Accuracy: 9.56%
Epoch [14/50], Train Loss: 2.3030, Validation Accuracy: 9.96%
Epoch [15/50], Train Loss: 2.3029, Validation Accuracy: 10.06%
Epoch [16/50], Train Loss: 2.3029, Validation Accuracy: 9.90%
Epoch [17/5

KeyboardInterrupt: 